<a href="https://colab.research.google.com/github/fassaud/Supply_Chain_Shipment_Pricing_Dataset_/blob/main/08_async_scheduling_soln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

#Scheduling Solution

In [ ]:
#inputs
num_workers = 8 #indexed with i
num_days = 7 #indexed with j
num_shifts = 3 #indexed with k

min_workers_per_shift = 2
max_shifts_per_day = 1
min_shifts_per_worker = 5
max_night_shifts = 2

In [ ]:
#optimization model
model = ConcreteModel() #create a model

#declare the decision variables
model.x = Var(range(num_workers), range(num_days), range(num_shifts), domain=Binary) #model.x[i,j,k] is the DV for the ith worker on day j, shift k

#Constraint: Each shift requires at least 2 nurses
#for each day j
    #for each shift k
        #add a constraint that the sum across the workers i >= 2
model.shiftshaveminworkers = ConstraintList()
for j in range(num_days): #for each day j
    for k in range(num_shifts): #for each shift k
        model.shiftshaveminworkers.add(expr = sum(model.x[i,j,k] for i in range(num_workers)) >= min_workers_per_shift)

#Constraint: no nurse works more than 1 shift per day
#for each worker i
    #for each day j
        #add a cosntraint that the sum across the shifts k is <= 1
model.maxshiftsperday = ConstraintList()
for i in range(num_workers):
    for j in range(num_days):
        model.maxshiftsperday.add(expr = sum(model.x[i,j,k] for k in range(num_shifts)) <= max_shifts_per_day)

#Constraint: at least 5 shifts for each nurse
model.minshifts = ConstraintList()
for i in range(num_workers):
  model.minshifts.add(expr = sum(model.x[i,j,k] for j in range(num_days) for k in range(num_shifts)) >= min_shifts_per_worker)

#Constraint: at most 2 night shifts per worker
model.nightshifts = ConstraintList()
for i in range(num_workers):
  model.nightshifts.add(expr = sum(model.x[i,j,k] for j in range(num_days) if k == 2) <= max_night_shifts)

model.Objective = Objective(expr = sum(model.x[i,j,k] for i in range(num_workers) for j in range(num_days) for k in range(num_shifts)), sense = minimize)

#model.pprint()

In [ ]:
opt = SolverFactory('cbc')
opt.options['ratioGap'] = 0.00
opt.options['seconds'] = 5
results = opt.solve(model, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -ratioGap 0.0 -seconds 5 -printingOptions all -import /tmp/tmpuoxmb2v2.pyomo.lp -stat=1 -solve -solu /tmp/tmpuoxmb2v2.pyomo.soln (default strategy 1)
ratioGap was changed from 0 to 0
seconds was changed from 1e+100 to 5
Option for printingOptions changed from normal to all
Presolve 93 (0) rows, 168 (0) columns and 560 (0) elements
Statistics for presolved model
Original problem has 168 integers (168 of which binary)
==== 0 zero objective 1 different
168 variables have objective of 1
==== absolute objective values 1 different
168 variables have objective of 1
==== for integers 0 zero objective 1 different
168 variables have objective of 1
==== for integers absolute objective values 1 different
168 variables have objective of 1
===== end objective counts


Problem has 93 rows, 168 columns (168 with objective) and 560 elements
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up,

In [ ]:
#printing out solution (with pandas)
print("total number of shifts assigned:", model.Objective())

total number of shifts assigned: 42.0


In [ ]:
schedule = [[[model.x[i,j,k]() for k in range(num_shifts)] for j in range(num_days)] for i in range(num_workers)]
import pandas as pd
pd.DataFrame(schedule, index = [f"worker{i}" for i in range(num_workers)], columns = [f"day{i}" for i in range(num_days)])

,day0,day1,day2,day3,day4,day5,day6
worker0,"[0.0, 1.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
worker1,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[1.0, 0.0, 0.0]"
worker2,"[1.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
worker3,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[1.0, 0.0, 0.0]"
worker4,"[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
worker5,"[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 1.0, 0.0]"
worker6,"[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
worker7,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]"


In [ ]:
#printing out solution alternative
print("total number of shifts assigned:", model.Objective())

#print out the solution by day
#for j in range(num_days):
#    for k in range(num_shifts):
#        for i in range(num_workers):
#            if model.x[i,j,k]() == 1:
#                print("day:", j, "shift:", k, "worker:", i)

#print out the solution by worker
for i in range(num_workers):
    for j in range(num_days):
        for k in range(num_shifts):
            if model.x[i,j,k]() == 1:
                print("worker:", i, "day:", j, "shift:", k)

total number of shifts assigned: 42.0
worker: 0 day: 0 shift: 1
worker: 0 day: 1 shift: 0
worker: 0 day: 2 shift: 0
worker: 0 day: 4 shift: 2
worker: 0 day: 5 shift: 1
worker: 0 day: 6 shift: 2
worker: 1 day: 0 shift: 2
worker: 1 day: 3 shift: 0
worker: 1 day: 4 shift: 1
worker: 1 day: 5 shift: 2
worker: 1 day: 6 shift: 0
worker: 2 day: 0 shift: 0
worker: 2 day: 1 shift: 1
worker: 2 day: 2 shift: 2
worker: 2 day: 3 shift: 0
worker: 2 day: 4 shift: 0
worker: 3 day: 0 shift: 2
worker: 3 day: 2 shift: 1
worker: 3 day: 4 shift: 0
worker: 3 day: 5 shift: 2
worker: 3 day: 6 shift: 0
worker: 4 day: 0 shift: 0
worker: 4 day: 1 shift: 0
worker: 4 day: 2 shift: 2
worker: 4 day: 3 shift: 1
worker: 4 day: 4 shift: 1
worker: 4 day: 5 shift: 0
worker: 5 day: 1 shift: 2
worker: 5 day: 2 shift: 0
worker: 5 day: 3 shift: 2
worker: 5 day: 5 shift: 0
worker: 5 day: 6 shift: 1
worker: 6 day: 1 shift: 2
worker: 6 day: 2 shift: 1
worker: 6 day: 3 shift: 2
worker: 6 day: 5 shift: 1
worker: 6 day: 6 shift: 1
